In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

### Load MNIST Data

In [14]:
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.00001
epochs        = 10
batch_size    = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout   = 0.75  # Dropout, probability to keep units

### 重みの定義

In [4]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

### 補足

In [8]:
print( "weights['wd1'].get_shape()              = ", weights['wd1'].get_shape() )
print( "weights['wd1'].get_shape().as_list()[0] = ", weights['wd1'].get_shape().as_list()[0] )

weights['wd1'].get_shape()              =  (3136, 1024)
weights['wd1'].get_shape().as_list()[0] =  3136


### 関数の定義 : conv2d
- convolution の計算用
- MNISTは、channel = 1　なので、strides[-1]=1となっている

In [9]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

### 関数の定義 : maxpool2d
- maxpooling の計算用
- MNISTは、channel = 1　なので、strides[-1]=1となっている

In [10]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

### 関数の定義 : conv_net
- 上の2つの関数を利用して、ネットワークを構築する関数
- [Conv-Pooling]->[Conv-Pooling]->[Dense-ReLu-Dropout]->[Dense]

In [11]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

### モデルの構築
- x : 入力、サイズ=28x28
- y : 出力、分類クラス10個

In [12]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Training / Validation / Test　の実行

In [13]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    # 初期化
    sess.run(init)
    # エポック毎のループ
    for epoch in range(epochs):       
        # バッチごとのループ
        for batch in range(mnist.train.num_examples//batch_size):
            # 今回使うデータの取得
            batch_x, batch_y = mnist.train.next_batch(batch_size)           
            # データを変数に登録し、最適化計算を実行
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            # 損失の計算
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            # Validation精度の計算
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Test精度の計算 at エポック毎
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 - Loss: 54748.8359 Validation Accuracy: 0.089844
Epoch  1, Batch   2 - Loss: 43119.6133 Validation Accuracy: 0.082031
Epoch  1, Batch   3 - Loss: 36533.1836 Validation Accuracy: 0.097656
Epoch  1, Batch   4 - Loss: 35378.2422 Validation Accuracy: 0.097656
Epoch  1, Batch   5 - Loss: 30975.5332 Validation Accuracy: 0.128906
Epoch  1, Batch   6 - Loss: 26903.9453 Validation Accuracy: 0.144531
Epoch  1, Batch   7 - Loss: 28709.2031 Validation Accuracy: 0.144531
Epoch  1, Batch   8 - Loss: 22860.2422 Validation Accuracy: 0.156250
Epoch  1, Batch   9 - Loss: 24623.6562 Validation Accuracy: 0.156250
Epoch  1, Batch  10 - Loss: 21807.0137 Validation Accuracy: 0.183594
Epoch  1, Batch  11 - Loss: 20895.4414 Validation Accuracy: 0.187500
Epoch  1, Batch  12 - Loss: 20512.5781 Validation Accuracy: 0.214844
Epoch  1, Batch  13 - Loss: 22621.8047 Validation Accuracy: 0.199219
Epoch  1, Batch  14 - Loss: 17083.2168 Validation Accuracy: 0.214844
Epoch  1, Batch  15 - Loss: 18823.

Epoch  1, Batch 127 - Loss:  3778.3208 Validation Accuracy: 0.628906
Epoch  1, Batch 128 - Loss:  3377.9502 Validation Accuracy: 0.652344
Epoch  1, Batch 129 - Loss:  3119.5581 Validation Accuracy: 0.648438
Epoch  1, Batch 130 - Loss:  3743.9907 Validation Accuracy: 0.640625
Epoch  1, Batch 131 - Loss:  3090.3997 Validation Accuracy: 0.640625
Epoch  1, Batch 132 - Loss:  3293.6270 Validation Accuracy: 0.648438
Epoch  1, Batch 133 - Loss:  3394.2703 Validation Accuracy: 0.640625
Epoch  1, Batch 134 - Loss:  2893.4290 Validation Accuracy: 0.636719
Epoch  1, Batch 135 - Loss:  2783.9048 Validation Accuracy: 0.652344
Epoch  1, Batch 136 - Loss:  4078.9277 Validation Accuracy: 0.652344
Epoch  1, Batch 137 - Loss:  2479.0984 Validation Accuracy: 0.660156
Epoch  1, Batch 138 - Loss:  3662.7834 Validation Accuracy: 0.656250
Epoch  1, Batch 139 - Loss:  2873.2966 Validation Accuracy: 0.660156
Epoch  1, Batch 140 - Loss:  3969.5303 Validation Accuracy: 0.667969
Epoch  1, Batch 141 - Loss:  3942.

Epoch  1, Batch 252 - Loss:  2577.4932 Validation Accuracy: 0.734375
Epoch  1, Batch 253 - Loss:  1368.8862 Validation Accuracy: 0.718750
Epoch  1, Batch 254 - Loss:  1552.6440 Validation Accuracy: 0.734375
Epoch  1, Batch 255 - Loss:  1915.4840 Validation Accuracy: 0.730469
Epoch  1, Batch 256 - Loss:  1756.4990 Validation Accuracy: 0.730469
Epoch  1, Batch 257 - Loss:  1869.6129 Validation Accuracy: 0.738281
Epoch  1, Batch 258 - Loss:  2442.6731 Validation Accuracy: 0.738281
Epoch  1, Batch 259 - Loss:  1515.7478 Validation Accuracy: 0.726562
Epoch  1, Batch 260 - Loss:  2045.5604 Validation Accuracy: 0.730469
Epoch  1, Batch 261 - Loss:  1920.1050 Validation Accuracy: 0.734375
Epoch  1, Batch 262 - Loss:  1964.8596 Validation Accuracy: 0.734375
Epoch  1, Batch 263 - Loss:  2537.9844 Validation Accuracy: 0.730469
Epoch  1, Batch 264 - Loss:  2131.8999 Validation Accuracy: 0.726562
Epoch  1, Batch 265 - Loss:  1845.2651 Validation Accuracy: 0.730469
Epoch  1, Batch 266 - Loss:  1551.

Epoch  1, Batch 376 - Loss:  1609.5973 Validation Accuracy: 0.753906
Epoch  1, Batch 377 - Loss:  1430.3203 Validation Accuracy: 0.746094
Epoch  1, Batch 378 - Loss:  1474.8303 Validation Accuracy: 0.746094
Epoch  1, Batch 379 - Loss:  1923.1919 Validation Accuracy: 0.746094
Epoch  1, Batch 380 - Loss:  1927.5447 Validation Accuracy: 0.746094
Epoch  1, Batch 381 - Loss:  2141.1865 Validation Accuracy: 0.738281
Epoch  1, Batch 382 - Loss:  1532.5752 Validation Accuracy: 0.746094
Epoch  1, Batch 383 - Loss:  1856.8071 Validation Accuracy: 0.746094
Epoch  1, Batch 384 - Loss:  1710.9951 Validation Accuracy: 0.746094
Epoch  1, Batch 385 - Loss:  1485.0824 Validation Accuracy: 0.750000
Epoch  1, Batch 386 - Loss:  1165.9871 Validation Accuracy: 0.750000
Epoch  1, Batch 387 - Loss:  1391.3376 Validation Accuracy: 0.742188
Epoch  1, Batch 388 - Loss:  1727.3655 Validation Accuracy: 0.746094
Epoch  1, Batch 389 - Loss:  1302.4858 Validation Accuracy: 0.746094
Epoch  1, Batch 390 - Loss:  1435.

Epoch  2, Batch  77 - Loss:  1013.7775 Validation Accuracy: 0.750000
Epoch  2, Batch  78 - Loss:  1263.1016 Validation Accuracy: 0.746094
Epoch  2, Batch  79 - Loss:  1030.5422 Validation Accuracy: 0.742188
Epoch  2, Batch  80 - Loss:  1526.7800 Validation Accuracy: 0.738281
Epoch  2, Batch  81 - Loss:  1393.2441 Validation Accuracy: 0.730469
Epoch  2, Batch  82 - Loss:  1065.3284 Validation Accuracy: 0.734375
Epoch  2, Batch  83 - Loss:   701.3774 Validation Accuracy: 0.738281
Epoch  2, Batch  84 - Loss:  1150.5233 Validation Accuracy: 0.730469
Epoch  2, Batch  85 - Loss:  1290.9980 Validation Accuracy: 0.742188
Epoch  2, Batch  86 - Loss:   947.7712 Validation Accuracy: 0.742188
Epoch  2, Batch  87 - Loss:  1376.4822 Validation Accuracy: 0.734375
Epoch  2, Batch  88 - Loss:  1095.2255 Validation Accuracy: 0.738281
Epoch  2, Batch  89 - Loss:  1046.4591 Validation Accuracy: 0.734375
Epoch  2, Batch  90 - Loss:  1186.2823 Validation Accuracy: 0.738281
Epoch  2, Batch  91 - Loss:  1300.

...

Epoch  9, Batch 296 - Loss:   187.3229 Validation Accuracy: 0.792969
Epoch  9, Batch 297 - Loss:   215.1403 Validation Accuracy: 0.792969
Epoch  9, Batch 298 - Loss:   219.8567 Validation Accuracy: 0.785156
Epoch  9, Batch 299 - Loss:   287.0724 Validation Accuracy: 0.785156
Epoch  9, Batch 300 - Loss:   285.1620 Validation Accuracy: 0.785156
Epoch  9, Batch 301 - Loss:   295.5450 Validation Accuracy: 0.785156
Epoch  9, Batch 302 - Loss:   151.0590 Validation Accuracy: 0.785156
Epoch  9, Batch 303 - Loss:   196.5801 Validation Accuracy: 0.785156
Epoch  9, Batch 304 - Loss:   335.6750 Validation Accuracy: 0.785156
Epoch  9, Batch 305 - Loss:   284.9827 Validation Accuracy: 0.785156
Epoch  9, Batch 306 - Loss:   329.3503 Validation Accuracy: 0.781250
Epoch  9, Batch 307 - Loss:   284.3076 Validation Accuracy: 0.781250
Epoch  9, Batch 308 - Loss:   233.0716 Validation Accuracy: 0.781250
Epoch  9, Batch 309 - Loss:   245.3256 Validation Accuracy: 0.781250
Epoch  9, Batch 310 - Loss:   233.

Epoch  9, Batch 420 - Loss:   324.4390 Validation Accuracy: 0.796875
Epoch  9, Batch 421 - Loss:   286.9277 Validation Accuracy: 0.792969
Epoch  9, Batch 422 - Loss:   347.3177 Validation Accuracy: 0.789062
Epoch  9, Batch 423 - Loss:   240.0217 Validation Accuracy: 0.789062
Epoch  9, Batch 424 - Loss:   209.4795 Validation Accuracy: 0.789062
Epoch  9, Batch 425 - Loss:   181.5150 Validation Accuracy: 0.792969
Epoch  9, Batch 426 - Loss:   244.6522 Validation Accuracy: 0.789062
Epoch  9, Batch 427 - Loss:   231.3608 Validation Accuracy: 0.789062
Epoch  9, Batch 428 - Loss:   346.9295 Validation Accuracy: 0.789062
Epoch  9, Batch 429 - Loss:   428.5261 Validation Accuracy: 0.785156
Epoch 10, Batch   1 - Loss:   290.2067 Validation Accuracy: 0.785156
Epoch 10, Batch   2 - Loss:   206.6433 Validation Accuracy: 0.785156
Epoch 10, Batch   3 - Loss:   341.0112 Validation Accuracy: 0.785156
Epoch 10, Batch   4 - Loss:   231.3266 Validation Accuracy: 0.785156
Epoch 10, Batch   5 - Loss:   353.

Epoch 10, Batch 122 - Loss:   251.2218 Validation Accuracy: 0.781250
Epoch 10, Batch 123 - Loss:   379.7596 Validation Accuracy: 0.781250
Epoch 10, Batch 124 - Loss:   287.6009 Validation Accuracy: 0.777344
Epoch 10, Batch 125 - Loss:   354.4144 Validation Accuracy: 0.769531
Epoch 10, Batch 126 - Loss:   306.6682 Validation Accuracy: 0.773438
Epoch 10, Batch 127 - Loss:   274.2377 Validation Accuracy: 0.773438
Epoch 10, Batch 128 - Loss:   169.6188 Validation Accuracy: 0.773438
Epoch 10, Batch 129 - Loss:   281.1483 Validation Accuracy: 0.769531
Epoch 10, Batch 130 - Loss:   184.9617 Validation Accuracy: 0.773438
Epoch 10, Batch 131 - Loss:   238.7098 Validation Accuracy: 0.773438
Epoch 10, Batch 132 - Loss:   214.0866 Validation Accuracy: 0.777344
Epoch 10, Batch 133 - Loss:   409.8860 Validation Accuracy: 0.781250
Epoch 10, Batch 134 - Loss:   141.3291 Validation Accuracy: 0.777344
Epoch 10, Batch 135 - Loss:   247.5882 Validation Accuracy: 0.777344
Epoch 10, Batch 136 - Loss:   367.

Epoch 10, Batch 245 - Loss:   256.1205 Validation Accuracy: 0.785156
Epoch 10, Batch 246 - Loss:   225.0006 Validation Accuracy: 0.777344
Epoch 10, Batch 247 - Loss:   181.8078 Validation Accuracy: 0.781250
Epoch 10, Batch 248 - Loss:   173.5596 Validation Accuracy: 0.777344
Epoch 10, Batch 249 - Loss:   240.1187 Validation Accuracy: 0.781250
Epoch 10, Batch 250 - Loss:   298.3789 Validation Accuracy: 0.785156
Epoch 10, Batch 251 - Loss:   311.6491 Validation Accuracy: 0.789062
Epoch 10, Batch 252 - Loss:   155.0046 Validation Accuracy: 0.789062
Epoch 10, Batch 253 - Loss:   235.4797 Validation Accuracy: 0.785156
Epoch 10, Batch 254 - Loss:   199.5551 Validation Accuracy: 0.781250
Epoch 10, Batch 255 - Loss:   324.4248 Validation Accuracy: 0.781250
Epoch 10, Batch 256 - Loss:   350.3820 Validation Accuracy: 0.781250
Epoch 10, Batch 257 - Loss:   351.0838 Validation Accuracy: 0.781250
Epoch 10, Batch 258 - Loss:   341.5720 Validation Accuracy: 0.777344
Epoch 10, Batch 259 - Loss:   194.

Epoch 10, Batch 367 - Loss:   264.2238 Validation Accuracy: 0.777344
Epoch 10, Batch 368 - Loss:   212.6080 Validation Accuracy: 0.777344
Epoch 10, Batch 369 - Loss:   166.0579 Validation Accuracy: 0.781250
Epoch 10, Batch 370 - Loss:   272.9427 Validation Accuracy: 0.781250
Epoch 10, Batch 371 - Loss:   230.2922 Validation Accuracy: 0.777344
Epoch 10, Batch 372 - Loss:   232.7191 Validation Accuracy: 0.781250
Epoch 10, Batch 373 - Loss:   183.4022 Validation Accuracy: 0.781250
Epoch 10, Batch 374 - Loss:   214.2393 Validation Accuracy: 0.777344
Epoch 10, Batch 375 - Loss:   270.0639 Validation Accuracy: 0.777344
Epoch 10, Batch 376 - Loss:   345.8258 Validation Accuracy: 0.773438
Epoch 10, Batch 377 - Loss:   363.4098 Validation Accuracy: 0.777344
Epoch 10, Batch 378 - Loss:   213.6180 Validation Accuracy: 0.773438
Epoch 10, Batch 379 - Loss:   377.0113 Validation Accuracy: 0.773438
Epoch 10, Batch 380 - Loss:   286.6047 Validation Accuracy: 0.773438
Epoch 10, Batch 381 - Loss:   233.